In [139]:
import numpy as np
import pandas as pd
import redis

In [140]:
# Connect to database

# redis-10581.c74.us-east-1-4.ec2.redns.redis-cloud.com:10581
# sk2XSE7YT6NWLFacA0N3CkasM97yF19a

hostname = "redis-10581.c74.us-east-1-4.ec2.redns.redis-cloud.com"
portnumber = 10581
password = "sk2XSE7YT6NWLFacA0N3CkasM97yF19a"

r = redis.StrictRedis(host= hostname, port= portnumber, password= password);


In [141]:
r.ping()

True

### Load the data from numpy.zip to database

In [142]:
np_file = np.load("stud_teacher_dataframe.npz", allow_pickle=True)
# file_np.files
x_values = np_file['arr_0']
col_names = np_file['arr_1']

# col_names
# x_values


In [143]:
df = pd.DataFrame(x_values, columns=col_names)
# df

for se in df['Facial_Features']:
    print(len(se))

512
512
512
512
512
512


In [144]:
df['name_role'] = df['Name'] + '@' + df['Role']
df

for se in df['Facial_Features']:
    print(len(se))

512
512
512
512
512
512


In [145]:
# making dictionary with name_role and facial features

records = df[['name_role', 'Facial_Features']].to_dict(orient="records")

len(records)
# records
# records[0]

6

### Save data in redis
- hashes
- keyname : "something"
- Discuss with guide for float32 and float64

In [146]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    # print(name_role)
    # print(vector)
    
    # convert numpy array into bytes to store in db easily
    vector_bytes = vector.astype(np.float64).tobytes()

    # save data in redis db
    r.hset(name='sharda', key=name_role, value=vector_bytes)

     # Retrieve and verify data from Redis
    stored_vector_bytes = r.hget(name='sharda', key=name_role)
    
    if stored_vector_bytes:
        print(f"Data for {name_role} successfully stored in Redis.")
    else:
        print(f"Failed to store data for {name_role}.")
    

Data for Ankur@Student successfully stored in Redis.
Data for Barack Obama@Teacher successfully stored in Redis.
Data for Inderpreet@Student successfully stored in Redis.
Data for Manasi@Student successfully stored in Redis.
Data for Mazhar@Student successfully stored in Redis.
Data for Sameer@Student successfully stored in Redis.


In [147]:
vector.dtype

dtype('float64')

## RETRIEVE RECORD FROM REDIS DB

In [148]:
retrieve_dict = r.hgetall(name="sharda")
retrieve_series = pd.Series(retrieve_dict)
retrieve_series

b'Manasi@Student'          b'UUUU\xc3\x80\xc7\xbf\x00\x00\x00\x80 Y\xed?U...
b'Barack Obama@Teacher'    b'ffff\x00h\xcc\xbf\xcd\xcc\xccL\x0c8\xf9?fff\...
b'Sameer@Student'          b'\x00\x00\x00\x80\xecf\xe2?\xcd\xcc\xcc\xcc/\...
b'Inderpreet@Student'      b'\x17]t\xd1]\x13\xf3?\xba\xe8\xa2\x8b\xdb\xcb...
b'Mazhar@Student'          b'\x00\x00\x00\xc0\x1b\xb1\xf6?\x00\x00\x00@M\...
b'Ankur@Student'           b'\x00\x00\x00\x80\x8c\xe8\xf9?\x00\x00\x00\x8...
dtype: object

In [149]:
# np.frombuffer(retrieve_series[b'Sameer@Student'], dtype=np.float32).shape
retrieve_series = retrieve_series.apply(lambda x : np.frombuffer(x, dtype=np.float64))
index = retrieve_series.index
index = list(map(lambda x : x.decode(), index))


In [150]:
retrieve_series.index = index
# index

In [151]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['name_role', 'facial_features']
retrieve_df

for idx, se in enumerate(retrieve_df["facial_features"]):
    print(f"Index: {idx}, Type: {type(se)}, Length: {len(se) if hasattr(se, '__len__') else 'N/A'}")


Index: 0, Type: <class 'numpy.ndarray'>, Length: 512
Index: 1, Type: <class 'numpy.ndarray'>, Length: 512
Index: 2, Type: <class 'numpy.ndarray'>, Length: 512
Index: 3, Type: <class 'numpy.ndarray'>, Length: 512
Index: 4, Type: <class 'numpy.ndarray'>, Length: 512
Index: 5, Type: <class 'numpy.ndarray'>, Length: 512


# FLUSH THE ENTIRE DATABASE

In [152]:
# Flush the entire database
# r.flushdb()

# REGISTRATION FORM

In [92]:
import cv2
from insightface.app import FaceAnalysis

In [93]:
# configure face analysis
faceapp = FaceAnalysis(name='buffalo_l', root='insightface_model', providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


## 1. COLLECT PERSON NAME AND ROLE

In [94]:
# python input
person_name = input("Enter your name : ")

trials = 3

for i in range(trials):
    role = input("""
    Please choose your role
    1. Student
    2. Teacher

    Enter number either 1 or 2
                    """)

    if role in ('1', '2'):
        if(role == '1'):
            role = "Student"
        else:
            role = "Teacher"
        break
    else:
        print("Invalid entry try again")
        if i == 3:
            print("Exceeds maximum trials")

key = person_name + "@" + role

Enter your name :  Veeru

    Please choose your role
    1. Student
    2. Teacher

    Enter number either 1 or 2
                     1


In [95]:
print("Name : ", person_name)
print("Role : ", role)
print("Key : ", key)

Name :  Veeru
Role :  Student
Key :  Veeru@Student


# 2. COLLECT PICTURE SAMPLES OF THE USER

In [116]:
# import cv2

# # cap = cv2.VideoCapture(1, cv2.CAP_DSHOW) # 0 is for default camera, 1 is for external camera
# cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)
# face_embeddings = []
# sample=0;

# if not cap.isOpened():
#     print("Camera not accessible")
#     exit()

# while True:
#     ret, frame = cap.read()

#     # get results from insightface model
#     results = faceapp.get(frame, max_num=1)
#     for res in results:
#         sample += 1
#         x1, y1, x2, y2 = res['bbox'].astype(int)
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

#     # facial features
#     embeddings = res['embedding']
#     face_embeddings.append(embeddings)

#     if sample >= 200:
#         break
    
#     if not ret:
#         print("Unable to read camera")
#         break
        
#     cv2.imshow("frame", frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):  # Ensuring proper key capture
#         break

# cap.release()
# cv2.destroyAllWindows()

In [18]:
# import cv2

# # cap = cv2.VideoCapture(1, cv2.CAP_DSHOW) # 0 is for default camera, 1 is for external camera
# cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# face_embeddings = []
# sample=0;

# if not cap.isOpened():
#     print("Camera not accessible")
#     exit()

# while True:
#     ret, frame = cap.read()

#     # get results from insightface model
#     # results = faceapp.get(frame, max_num=1)
#     # for res in results:
#     #     sample += 1
#     #     x1, y1, x2, y2 = res['bbox'].astype(int)
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

#     # # facial features
#     # embeddings = res['embedding']
#     # face_embeddings.append(embeddings)

#     if sample >= 200:
#         break
    
#     if not ret:
#         print("Unable to read camera")
#         break
        
#     cv2.imshow("frame", frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):  # Ensuring proper key capture
#         break

# cap.release()
# cv2.destroyAllWindows()

Unable to read camera


In [ ]:
# cap = cv2.VideoCapture(0) # 0 is for default camera, 1 is for external camera

# while True:
#     ret, frame = cap.read()  # ret is true or false for read value
    
#     if ret == False:
#         print("Unable to read camera")
#         break
        
#     cv2.imshow("frame", frame)
#     if cv2.waitkey(1) == ord('q'): # triggers only when q is pressed to stop the camera
#         break

# cap.release()
# cv2.destroyAllWindows()

In [18]:
# frame
# len(face_embeddings)

In [8]:
# import cv2
# cap = cv2.VideoCapture(1, cv2.CAP_DSHOW)
# face_embeddings = []

# if not cap.isOpened():
#     print("Camera not accessible")
#     exit()

# while True:
#     ret, frame = cap.read()

#     # get results from insightface model
#     results = faceapp.get(frame, max_num=1)
#     for res in results:
#         x1, y1, x2, y2 = res['bbox'].astype(int)
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

#      # facial features
#     embeddings = res['embedding']
#     face_embeddings.append(embeddings)
    
#     if not ret:
#         print("Unable to read camera")
#         break

#     cv2.imshow("Camera Feed", frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


In [22]:
# import time

# # Variables
# face_embeddings = []
# sample = 0
# camera_index = 1  # Replace this with the correct camera index
# cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)

# if not cap.isOpened():
#     print(f"Camera at index {camera_index} is not accessible. Try another index.")
#     exit()

# # Camera settings
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# time.sleep(2)  # Allow camera to initialize
# print("Starting camera feed... Press 'q' to exit.")

# while True:
#     ret, frame = cap.read()

#     if not ret or frame is None:
#         print(f"Failed to capture frame from camera index {camera_index}.")
#         break

#     # Preprocess the frame (resize for consistent input)
#     frame_resized = cv2.resize(frame, (640, 480))

#     # Get results from the model
#     results = faceapp.get(frame_resized, max_num=1)
    
#     for res in results:
#         sample += 1
#         x1, y1, x2, y2 = res['bbox'].astype(int)
#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

#         # Extract facial embeddings
#         embeddings = np.array(res['embedding'])

#         # Validate embedding size
#         if embeddings.shape[0] == 512:
#             face_embeddings.append(embeddings)
#         else:
#             print(f"Unexpected embedding size: {embeddings.shape}. Skipping.")

#     if sample >= 50:
#         break

#     cv2.imshow("Camera Feed", frame_resized)

#     if cv2.waitKey(1) == ord('q'):  # Press 'q' to quit
#         break

# cap.release()
# cv2.destroyAllWindows()


Starting camera feed... Press 'q' to exit.


In [49]:
cap = cv2.VideoCapture(1,cv2.CAP_DSHOW) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),1)
        
        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
        
    if sample >= 50: 
        break
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): # this is trigger only when I press letter q (lowercase q) in my keyboard
        break
        
cap.release()
cv2.destroyAllWindows()

### CHECK FOR CAMERA INDEX

In [96]:
import cv2

print("Checking available cameras...")
for index in range(5):  # Check indexes from 0 to 4
    cap = cv2.VideoCapture(index, cv2.CAP_DSHOW)
    if cap.isOpened():
        print(f"Camera found at index {index}")
        cap.release()
    else:
        print(f"No camera found at index {index}")


Checking available cameras...
Camera found at index 0
Camera found at index 1
No camera found at index 2
No camera found at index 3
No camera found at index 4


In [97]:
import cv2
import time

camera_index = 1  # Replace with correct index
cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)

if not cap.isOpened():
    print(f"Camera at index {camera_index} is not accessible.")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

time.sleep(2)  # Allow camera to initialize
print("Starting camera feed... Press 'q' to exit.")

while True:
    ret, frame = cap.read()

    if not ret or frame is None:
        print(f"Failed to capture frame from camera index {camera_index}.")
        break

    cv2.imshow("Camera Feed", frame)

    if cv2.waitKey(1) == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()


Starting camera feed... Press 'q' to exit.


In [98]:
import cv2
import time

face_embeddings = []
sample=0

camera_index = 1  # Replace this with the correct index found from the previous step
cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)

if not cap.isOpened():
    print(f"Camera at index {camera_index} is not accessible. Try another index.")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

time.sleep(2)  # Allow camera to initialize
print("Starting camera feed... Press 'q' to exit.")

while True:
    ret, frame = cap.read()

    # get results from insightface model
    results = faceapp.get(frame, max_num=1)
    print(results)
    
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)

    if sample >= 50:
        break
    
    if not ret or frame is None:
        print(f"Failed to capture frame from camera index {camera_index}.")
        break

    cv2.imshow("Camera Feed", frame)

    if cv2.waitKey(1) == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()


Starting camera feed... Press 'q' to exit.
[{'bbox': array([204.14468,  53.99095, 406.81747, 328.9954 ], dtype=float32), 'kps': array([[253.04462, 162.03201],
       [354.12598, 151.15515],
       [308.15665, 200.08223],
       [275.88232, 266.94302],
       [352.2508 , 257.91876]], dtype=float32), 'det_score': np.float32(0.81554836), 'landmark_3d_68': array([[ 1.88108887e+02,  1.55336685e+02,  1.52371490e+02],
       [ 1.95606995e+02,  1.85656540e+02,  1.53655273e+02],
       [ 2.03501114e+02,  2.17529007e+02,  1.51664062e+02],
       [ 2.12744339e+02,  2.48265213e+02,  1.43502930e+02],
       [ 2.26095123e+02,  2.77396393e+02,  1.22061554e+02],
       [ 2.42420395e+02,  2.98703400e+02,  1.03369164e+02],
       [ 2.61380493e+02,  3.13946777e+02,  8.85590439e+01],
       [ 2.85441559e+02,  3.24223450e+02,  6.97049866e+01],
       [ 3.23976044e+02,  3.29040894e+02,  6.05154190e+01],
       [ 3.58108795e+02,  3.16754822e+02,  7.68210297e+01],
       [ 3.79200989e+02,  2.97523590e+02,  1.

In [153]:
len(face_embeddings)
# len(frame)
# len(frame)
# results

50

# 3. OPTIMISE DATA & SAVE IT TO THE REDIS DATABASE

In [154]:
# list -> array for mean calculation
x_mean = np.asarray(face_embeddings).mean(axis=0)

# each sample has 512 dimensions
x_mean.shape

(512,)

In [155]:
# x_mean
print(f"x_mean shape before storing: {x_mean.shape}")
print(f"x_mean size in bytes: {x_mean.nbytes} bytes")  # Should be 4096 bytes (512 * 8)
print(f"x_mean dtype: {x_mean.dtype}")


x_mean shape before storing: (512,)
x_mean size in bytes: 2048 bytes
x_mean dtype: float32


In [156]:
# array -> bytes , for storing into redis db
# x_mean_bytes = x_mean.tobytes()
x_mean_bytes = x_mean.astype(np.float64).tobytes()

print(f"Length of x_mean_bytes: {len(x_mean_bytes)} bytes")  # Should also be 4096 bytes

Length of x_mean_bytes: 4096 bytes


In [157]:
# stored_data = r.hget(name='sharda', key=key)
# print(f"Size of data retrieved from Redis: {len(stored_data)} bytes")  # Should still be 4096 bytes

In [158]:
# save key and facial bytes into redis db
r.hset(name='sharda', key=key, value=x_mean_bytes)

1